In [724]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy.optimize import newton

In [873]:
def rate(N,T,r0):
    R = np.zeros([N+1,N+1])
    R[0,0] = r0
    delta = T/N
    for i in range(1,N+1):
        for j in range(0,i+1):
            R[j,i] = 0.0225*math.exp(0.45*(2*j-i)*math.sqrt(delta))
    return R

In [874]:
R = rate(36,3,0.0225)

In [875]:
def pi(pi,N,T,R):
    p = np.zeros([N+1,N+1])
    p[0,0]=1
    delta = T/N
    for i in range(1, N+1):
        p[0,i] = 1/(1+R[0,i-1]*delta)*(1-pi)*p[0,i-1]
        for j in range(1,i):
            p[j,i] = 1/(1+R[j-1,i-1]*delta)*pi*p[j-1,i-1] + 1/(1+R[j,i-1]*delta)*(1-pi)*p[j,i-1]
        p[i,i] = 1/(1+R[i-1,i-1]*delta)*pi*p[i-1,i-1]
    return p
    

In [876]:
PI = pi(0.5,36,3,R)

In [877]:
PSvalue = np.sum(PI,axis = 0)

In [878]:
interest = [11.25,11.25,12.5,12.5,12.5,13.75]
ps = [PSvalue[6],PSvalue[12],PSvalue[18],PSvalue[24],PSvalue[30],PSvalue[36]]

In [879]:
bond = sum(np.array(interest)*np.array(ps))+1000*PSvalue[36]
bond
sum(np.array(interest)*np.array(ps))

70.5195420279756

In [907]:
bond = np.zeros([37,37])
bond[:,36] = 1013.75
for t in range(35,-1,-1):
    for j in range(0,t+1):
        if t%6 == 0 and t!=0:
            r = rate(36-t,3-t/12,R[j,t])
            PI_ = pi(0.5,36-t,3-t/12,r)
            PSvalue = np.sum(PI_,axis = 0)
            ps = [PSvalue[6*i] for i in range(0,int(6-t/6)+1)]
            bond[j,t]=min(1000+interest[int(t/6-1)],0.5*(bond[j,t+1]/(1+R[j,t]*(1/12)))+0.5*(bond[j+1,t+1]/(1+R[j,t]*(1/12)))+interest[int(t/6-1)])
        else:
            bond[j,t]=0.5*(bond[j,t+1]/(1+R[j,t]*(1/12)))+0.5*(bond[j+1,t+1]/(1+R[j,t]*(1/12)))
bond[0,0]

        

986.2134383721445

In [931]:
d = 0
for t in [6,12,18,24,30]:
    for j in range(0,t+1):
        r = rate(36-t,3-t/12,R[j,t])
        PI_ = pi(0.5,36-t,3-t/12,r)
        PSvalue = np.sum(PI_,axis = 0)
        ps = [PSvalue[6*i] for i in range(1,int(6-t/6)+1)]
        b = sum(np.array(interest[int(t/6):6])*np.array(ps))+1000*ps[-1]
        if b >= 1000:
            d = d + (1000-b)*PI[j,t]
callablebond = bond+d  
callablebond


array([[ 978.84470814,  987.45912531,  994.00298257, ..., 1006.33517869,
        1006.36111649, 1006.38126977],
       [  -7.36873023,  973.92859137,  984.19140989, ..., 1006.32150497,
        1006.35513756, 1006.38126977],
       [  -7.36873023,   -7.36873023,  967.85610442, ..., 1006.30377494,
        1006.3473849 , 1006.38126977],
       ...,
       [  -7.36873023,   -7.36873023,   -7.36873023, ...,  751.52964314,
         884.71956916, 1006.38126977],
       [  -7.36873023,   -7.36873023,   -7.36873023, ...,   -7.36873023,
         854.04871712, 1006.38126977],
       [  -7.36873023,   -7.36873023,   -7.36873023, ...,   -7.36873023,
          -7.36873023, 1006.38126977]])

In [910]:
def libor(N,T,r0):
    R = np.zeros([N+1,N+1])
    R[0,0] = r0
    delta = T/N
    for i in range(1,N+1):
        for j in range(0,i+1):
            R[j,i] = 0.025*math.exp(0.4*(2*j-i)*math.sqrt(delta))
    return R

In [911]:
t = libor(60,5,0.025)
tree = pd.DataFrame(t).to_excel('Q3_tree.xlsx')
ad = pi(0.5,60,5,t)
AD = pd.DataFrame(ad).to_excel('Q3_AD.xlsx')
psvalue = np.sum(ad, axis = 0)


In [929]:
caps = []  #[0.0011179531871522523, 0.0018334750020600956, 0.003242789500628622]
for y in [9,21,57]:
    caplet = 0
    ps = []
    pp = np.zeros([3,58])
    for s in range(y+1):
        p = np.zeros([4,4])
        p[:,3]= 1
        for i in range(2,-1,-1):
            for j in range(0,i+1):
                p[j,i] = 0.5*(p[j+1,i+1]/(1+t[s+j,y+i]*(1/12)))+0.5*(p[j,i+1]/(1+t[s+j,y+i]*(1/12)))
        caplet = caplet+0.25*max(((1/p[0,0]-1)/(0.25))-0.025,0)*ad[s,y]*p[0,0]
    caps.append(caplet)
print(caps)

[0.0011179531871522523, 0.0018334750020600956, 0.003242789500628622]


In [913]:
forward = []  # [0.026734974120046218, 0.02876654088315611, 0.03423084474447702]
for y in [9,21,57]:
    forward.append((psvalue[y]/psvalue[y+3]-1)*4)  

In [915]:
import scipy as sp
import pylab as pl
from scipy.optimize import newton
def bsfunc(i,y,sigma):
    d1 = (math.log(forward[i]/0.025)+0.5*sigma**2*(y/12))/(sigma*math.sqrt(y/12))
    d2 = (math.log(forward[i]/0.025)-0.5*sigma**2*(y/12))/(sigma*math.sqrt(y/12))
    bscap = psvalue[y+3]*0.25*(forward[i]*norm.cdf(d1)-0.025*norm.cdf(d2))
    return bscap



In [916]:

for y,i in zip([9,21,57],[0,1,2]):
    error = (lambda sigma: bsfunc(i,y,sigma)-caps[i])
    sigma = newton(error,0.2,tol = 0.0000000148)
    print(sigma)
    

0.4114361803000908
0.4041870956017511
0.39358478572265543


In [917]:
#m = month, swaption = total month of swaption, swap= total month of swap
def CPB(rate,T,N,R,swaption,swap,m,pi):
    cpb = np.zeros([swaption+swap+1,swaption+swap+1])
    delta = T/N
    for j in range(0,swap+swaption+1):
        cpb[j,swap+swaption] = 1+rate*delta*m
    for i in range(swap+swaption-1,swaption-1,-1):
        for j in range(0,i+1):
            cpb[j,i] = cpb[j,i+1]/(1+R[j,i]*delta)*(1-pi)+cpb[j+1,i+1]/(1+R[j,i]*delta)*pi
            if i%m == 0 and i != swaption:
                cpb[j,i] = cpb[j,i]+rate*delta*m
    return cpb
                
    

In [918]:
def swaption(CPB, PI, swaption):
    s = 0
    for j in range(swaption+1):
        s = s + max(0, CPB[j,swaption]-1)*PI[j,swaption]
    return s

In [919]:
cpb = CPB(0.025,5,60,t,36,24,6,0.5)
s = swaption(cpb,ad,36)

In [920]:
s

0.007800482390803046

In [796]:
pd.DataFrame(cpb).to_excel('Q3-d.xlsx')

In [836]:
def swfunc(s,y,sigma):
    d1 = (math.log(s/0.025)+0.5*sigma**2*(y/12))/(sigma*math.sqrt(y/12))
    d2 = (math.log(s/0.025)-0.5*sigma**2*(y/12))/(sigma*math.sqrt(y/12))
    Swaption = 0.5*(psvalue[42]+psvalue[48]+psvalue[54]+psvalue[60])*(0.025*norm.cdf(-d2)-s*norm.cdf(-d1))
    return Swaption

sw = (psvalue[36]-psvalue[60])/(0.5*(psvalue[42]+psvalue[48]+psvalue[54]+psvalue[60]))

In [837]:
error = (lambda sigma: swfunc(sw,36,sigma)-s)
sigma = newton(error,0.1,tol = 0.0000000148)
print(sigma)

0.39644893754409666


In [863]:
Bermudan = 0
s = np.zeros([37,37])
for i in range(36,-1,-1):
    if i == 36:
        cpb = CPB(0.025,5,60,t,i,60-i,6,0.5)
        for j in range(i+1):
            s[j,i] = max(0, cpb[j,i]-1)
    elif i%6 == 0 :
        cpb = CPB(0.025,5,60,t,i,60-i,6,0.5)
        for j in range(i+1):
            s[j,i] = max(0.5*s[j,i+1]/(1+t[j,i]*(1/12))+0.5*s[j+1,i+1]/(1+t[j,i]*(1/12)), cpb[j,i]-1)
    else:
        for j in range(i+1):
            s[j,i] = 0.5*s[j,i+1]/(1+t[j,i]*(1/12))+0.5*s[j+1,i+1]/(1+t[j,i]*(1/12))
                                                        
print(s[0,0])  #ans 0.012385259510417232

0.012385259510417232
